<img src="images/datastaxdevs_banner.png" width="600" height="200">

# Algorithm 3: Random Forest
------
<img src="images/drinkWine.jpeg" width="300" height="500">


#### Dataset: https://archive.ics.uci.edu/ml/datasets/Wine+Quality

## What are we trying to learn from this dataset? 

### Can Random Forest be used to guess a wine's rating score from its attributes?

In [ ]:
import os
import pandas
from pyspark.sql import SparkSession
#
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#
from pyspark.ml.classification import RandomForestClassifier
#
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
#
from dotenv import load_dotenv, find_dotenv

from tools import showDF, examineCassandraTable

In [ ]:
# read .env file for connection params
dotenv_file = find_dotenv('.env')
load_dotenv(dotenv_file)
astraUsername = os.environ['ASTRA_DB_CLIENT_ID']
astraPassword = os.environ['ASTRA_DB_CLIENT_SECRET']
astraSecureConnect = os.environ['ASTRA_DB_SECURE_BUNDLE_PATH']
astraKeyspace = os.environ['ASTRA_DB_KEYSPACE']

## Inspect input data: Table(s)

### Connect to Cassandra

In [ ]:
cloud_config = {
    'secure_connect_bundle': '/home/jovyan/' + astraSecureConnect
}
auth_provider = PlainTextAuthProvider(username=astraUsername, password=astraPassword)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

### Set keyspace 

In [ ]:
session.set_keyspace(astraKeyspace)

### Examine table `wines` (structure and contents)

In [ ]:
print(examineCassandraTable(session, astraKeyspace, 'wines'))

### What do these 12 columns represent: 

* **Fixed acidity**
* **Volatile acidity**
* **Citric Acid**
* **Residual Sugar** 
* **Chlorides**
* **Free sulfur dioxide**     
* **Total sulfur dioxide**
* **Density** 
* **pH**
* **Sulphates**
* **Alcohol**
* **Quality**

<img src="images/whiteAndRed.jpeg" width="300" height="300">

# Machine Learning with Apache Cassandra & Apache Spark
<img src="images/sparklogo.png" width="150" height="200">

### Create a Spark session that is connected to the database. From there load each table into a Spark Dataframe and take a count of the number of rows in each.

In [ ]:
spark = SparkSession \
    .builder \
    .appName('demo') \
    .master('local') \
    .config( \
        'spark.cassandra.connection.config.cloud.path', \
        'file:' + '/home/jovyan/' + astraSecureConnect) \
    .config('spark.cassandra.auth.username', astraUsername) \
    .config('spark.cassandra.auth.password', astraPassword) \
    .getOrCreate()

wineDF = spark \
    .read \
    .format('org.apache.spark.sql.cassandra') \
    .options(table='wines', keyspace=astraKeyspace) \
    .load()

print ('Table Wine Row Count:')
print (wineDF.count())

In [ ]:
showDF(wineDF)

#### Let's filter out only wines that have been rated 6.0 or higher and create a new dataframe with that information 

In [ ]:
wine6DF = wineDF.filter('quality > 5')
showDF(wine6DF)

#### Create Vector with all elements of the wine 

In [ ]:
assembler = VectorAssembler(
    inputCols=['alcohol', 'chlorides', 'citricacid', 'density', 'fixedacidity', 'ph',
               'freesulfur', 'sugar', 'sulphates', 'totalsulfur', 'volatileacidity'],
    outputCol='features',
)

trainingData = assembler.transform(wine6DF)

labelIndexer = StringIndexer(inputCol='quality', outputCol='label', handleInvalid='keep')
trainingData1 = labelIndexer.fit(trainingData).transform(trainingData)

showDF(trainingData1)
print(trainingData1.count())

We will be training a model with Random Forest, and because of this we need to split up our dataset into a training and test set. Will split 80/20. 

In [ ]:
# Split the data into train and test
splits = trainingData1.randomSplit([0.8, 0.2], 1234)
train = splits[0]
test = splits[1]

print ('Train Dataframe Row Count:')
print (train.count())
print ('Test Datafram Row Count:')
print (test.count())

In [ ]:
rf = RandomForestClassifier(labelCol='label', featuresCol='features', numTrees=10)

model = rf.fit(train)

predictions = model.transform(test)
print (predictions.count())
showDF(predictions)

In [ ]:
showDF(predictions.select('quality', 'label', 'prediction', 'probability'), limitRows=15)

### We can now use the `MulticlassClassificationEvaluator` to evalute the accuracy of our predictions. 

In [ ]:
# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(
    labelCol='label',
    predictionCol='prediction',
    metricName='accuracy',
)
accuracy = evaluator.evaluate(predictions)
print('Test set accuracy = %.4f' % accuracy)

## Example model usage

_Note: in real life, your input is probably massive (as opposed to a single row); also, it is likely read from the database._

In [ ]:
def predict_wine_quality(**kwargs):
    input_df = pandas.DataFrame([kwargs])
    spark_input = spark.createDataFrame(input_df)
    spark_with_features = assembler.transform(spark_input)
    predicted = model.transform(spark_with_features)
    collected = predicted.collect()
    #
    return {
        'prediction': collected[0].prediction,
        'probability': list(collected[0].probability),
    }


In [ ]:
predict_wine_quality(
    alcohol=15.0,
    chlorides=0.017,
    citricacid=0.31,
    density=0.99100,
    fixedacidity=5.1,
    freesulfur=12.0,
    ph=4.14,
    sugar=1.90,
    sulphates=0.50,
    totalsulfur=103.0,
    volatileacidity=0.170,
)

#### Stop the Spark session

In [ ]:
spark.stop()